In [1]:
# -*- coding: utf-8 -*-
import librosa
import requests
from gerapy_auto_extractor.extractors import *
import os
import csv
import glob
import urllib.parse
import numpy as np
# from skimage import transform as tf
from moviepy.editor import *
from moviepy.video.tools.drawing import color_gradient
from random import choice

font_path = './font/kaiti.ttf' # 加载字体配置文件
log_path  = "logs/生成过内容的记录表单.csv" # 文件保存路径

# 自定义处理数据函数
from EveryWebsitFuction import *
from WordProcess import *
from AudioProcess import *
from ImageProcess import * 
from VideoProcess import *
from FileProcess import  *

# 抓取文章基本信息记录

In [2]:
# 需要一键转换的文章链接
Url = "https://haopinang.com/27793.html"
# 提取站点信息用于爬虫脚本
WebSite = urllib.parse.urlsplit(Url).netloc
# 用爬虫extract_detail提取文章的标题
Title = extract_detail(requests.get(Url,headers=headers,proxies=proxies,verify=False).text)["title"]
# 构建文章制作信息日志
data_row = [WebSite,Title,Url]
write_csv(log_path, data_row)

# 抓取文章的正文、标题、图片

In [3]:
# 访问该页面
response = requests.get(Url,headers=headers,proxies=proxies,verify=False)
# 根据固定的方法获取标题、正文、图片link
if WebSite == "haopinang.com":
    title,content,imglist = haopinang(response)

# 这里文字需要自己处理一下，避免敏感词汇

In [5]:
title

'台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感'

In [40]:
title = "台下观众晃到嗨翻,Ant”弹跳打节拍,性感DJ“张蚂蚁,S2O最动感"

In [6]:
content

'好皮囊：将来或许容颜会消逝，但照片的你永远不会老去“2020 S2O Taiwan Mix Edition 泰国泼水音乐节”当天除了国际知名DJ外，在“WaterBOOM湿身电音派对”区域里也有许多正妹DJ轮番上阵，这次JKF女郎张蚂蚁 Ant也上台一起嗨啦！当天张蚂蚁 Ant穿着桃红色比基尼展现好身材，和帅气的短发造型成反差，是S2O最动感！张蚂蚁 Ant当天在台上刷碟时，随着节奏摇摆身体，胸前的美乳也跟着弹跳打节拍，这样的爆乳派对也难怪台下观众都一起晃到头晕了，其他JKF女郎也上台一起同乐，乳力满点的画面可以说是S2O最值得看的一部分。张蚂蚁 Ant现在转型成专业DJ后，也不吝啬分享火辣身材，Instagram内一样满满的爆乳美照，只是好怀念舞后张蚂蚁 Ant的电臀啊！希望哪天还有机会看到她再摇起来。IG：dj_ant_tw好了，美图时刻开始：未经允许不得转载：好皮囊 » 台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感'

In [42]:
content = "“2020 S2O Taiwan Mix Edition 泰国泼水音乐节”当天除了国际知名DJ外，在“WaterBOOM湿身电音派对”区域里也有许多正妹DJ轮番上阵，这次JKF女郎张蚂蚁 Ant也上台一起嗨啦！当天张蚂蚁 Ant穿着桃红色比基尼展现好身材，和帅气的短发造型成反差，是S2O最动感！张蚂蚁 Ant当天在台上刷碟时，随着节奏摇摆身体，胸前的美乳也跟着弹跳打节拍，这样的爆乳派对也难怪台下观众都一起晃到头晕了，其他JKF女郎也上台一起同乐，乳力满点的画面可以说是S2O最值得看的一部分。张蚂蚁 Ant现在转型成专业DJ后，也不吝啬分享火辣身材，Instagram内一样满满的爆乳美照，只是好怀念舞后张蚂蚁 Ant的电臀啊！希望哪天还有机会看到她再摇起来。"

In [43]:
# 如果文稿过长可以使用百度AI提取摘要
# https://ai.baidu.com/tech/nlp_apply/news_summary
# content = make_summary(title,content)
content

'“2020 S2O Taiwan Mix Edition 泰国泼水音乐节”当天除了国际知名DJ外，在“WaterBOOM湿身电音派对”区域里也有许多正妹DJ轮番上阵，这次JKF女郎张蚂蚁 Ant也上台一起嗨啦！当天张蚂蚁 Ant穿着桃红色比基尼展现好身材，和帅气的短发造型成反差，是S2O最动感！张蚂蚁 Ant当天在台上刷碟时，随着节奏摇摆身体，胸前的美乳也跟着弹跳打节拍，这样的爆乳派对也难怪台下观众都一起晃到头晕了，其他JKF女郎也上台一起同乐，乳力满点的画面可以说是S2O最值得看的一部分。张蚂蚁 Ant现在转型成专业DJ后，也不吝啬分享火辣身材，Instagram内一样满满的爆乳美照，只是好怀念舞后张蚂蚁 Ant的电臀啊！希望哪天还有机会看到她再摇起来。'

# 图片裁剪处理

In [7]:
# 判断如果没有该数据的文件夹就创建
dirs = 'materials/' + title + "/web_jpg/"
if not os.path.exists(dirs):
    os.makedirs(dirs)
imglist = RequestGetImage(title) + imglist # 将浏览器百度搜索的素材和文章中的素材整理在一起
    
# 保存图片并按比例裁剪
i = 1
jpg_list = [] # 用于保存图片的文件名成
for url in imglist:   #读取并保存到本地
    html = requests.get(url,headers=headers,proxies=proxies,verify=False)
    # 将抓取的图片保存到web_jpg
    with open('materials/'+ title + "/web_jpg/" + str(i) + ".jpg", 'wb') as file:
        file.write(html.content)
    file.close()
    CutImage(i,title) # 裁剪 （与去水印二选一）
#     CleanLogo(i,title,240) # 去水印
    ChangeImage(i,title) # 裁剪图片
    jpg_list.append("{}.jpg".format(i))
    i += 1

按比例裁剪完毕，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
图片按照按视频寸尺裁剪，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
按比例裁剪完毕，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
图片按照按视频寸尺裁剪，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
按比例裁剪完毕，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
图片按照按视频寸尺裁剪，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
按比例裁剪完毕，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
图片按照按视频寸尺裁剪，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
按比例裁剪完毕，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
图片按照按视频寸尺裁剪，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
按比例裁剪完毕，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
图片按照按视频寸尺裁剪，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
按比例裁剪完毕，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/clean_jpg/
图片按照按视频寸尺裁剪，保存到： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“

# 音频和字幕处理

In [8]:
# 获取字幕行数和字幕
word_len , strs = clean_word(title + content)
# 去除无用的空list避免报错
strs_list = strs.split("\n")
strs_list = [i for i in strs_list if i != ""] 
#生成MP3文件
StrToMp3(title,content)
# 获取MP3时长
mp3_duration = librosa.get_duration(filename='materials/'+ title + "/data/" +  title + '.mp3')

mp3文件已经生成完毕，在目录： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/


# 制作

## 删减无效图片

In [9]:
# 图片目录中必须要有cover.jpg
file_dir = "materials/" + title + "/clean_jpg"
jpg_list = [files for files in os.walk(file_dir)][0][2]

## 生成封面图片

In [10]:
# 这里要自己选择一张图片改名cover.jpg
MakeCoverImage(title)

封面图片制作完毕，目录： materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/cover.jpg


## 生成封面视频

In [11]:
# 根据制作好的cover.jpg生成封面视频,并添加文章标题
MakeCoverVideoAndTitle(title,font_path)

Moviepy - Building video ./materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/cover.mp4.
Moviepy - Writing video ./materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/cover.mp4



Moviepy - Done !
Moviepy - video ready ./materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/cover.mp4


## 合成全部视频

In [12]:
cover_mp4 = "./materials/" + title + "/data/cover.mp4"  # 合成的封面
strat_mp4 = "./materials_base/start.mp4"  # 自制片头
end_mp4 = "./materials_base/end.mp4"  # 自制片尾
logo_path = "materials_base/logo.png" # 自制水印

CompositeVideo(title,mp3_duration,jpg_list,font_path,strs_list,logo_path,cover_mp4,strat_mp4,end_mp4)

图片拼接函数处理完毕！
字幕拼接函数处理完毕！
视频添加水印完毕！
在背景视频中处理字幕和拼接图片完毕！
解说音频合成在视频音轨完毕！
视频内容拼接完毕！
Moviepy - Building video materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感.mp4.
MoviePy - Writing audio in 台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感.mp4



Moviepy - Done !
Moviepy - video ready materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感.mp4
背景视频中添加轮播图片、字幕、音频解说合成完毕，保存目录： ./materials/台下观众晃到嗨翻,Ant”爆乳弹跳打节拍,性感DJ“张蚂蚁,S2O最动感/data/


# 删除多余的文件

In [18]:
DeleteFiles()

# 调试部分

In [ ]:
# 纵向图片向上移动  'center','top'
def VerticalImageUp():
    fl = lambda gf,t:gf(t)[int(image_speed*t):int(image_speed*t)+ 720,:] 
    
    return fl,('center','top')
# 纵向图片向左移动  'center','top'
def VerticalImageLeft():
    fl = lambda gf,t:gf(t)[:,int(image_speed*t):int(image_speed*t)+ 720] 
    return fl,('center','top')


# 横向图片向上移动  'center','top'
def HorizontalUp():
    fl = lambda gf,t:gf(t)[int(image_speed*t):int(image_speed*t)+ 720,:] 
    return fl,('center','top')
# 横向图片向左移动  'center','top'
def HorizontalLeft():
    fl = lambda gf,t:gf(t)[:,int(image_speed*t):int(image_speed*t)+ 1280] 
    return fl,('center','top')

# 创建背景视频
bak_movei = VideoFileClip('materials_base/base1280_720.mp4')

# 读取图片
jpg_path = "materials/45岁林志玲身体亮红灯！被曝秘密入院做手术/clean_jpg/5.jpg"
img = Image.open(jpg_path)
w , h = img.size[0],img.size[1]  # 图片高度和宽度后面进行判断使用
print( w , h )
every_part_duration_time = 10
start_time = 0
clip_image = (
            ImageClip(jpg_path)
                .set_duration(every_part_duration_time)  # 图片持续时间
                .set_start(start_time)
        )

# 图片移动的速度
image_speed = 10
# VerticalImageUp,VerticalImageLeft,HorizontalUp,HorizontalLeft

list1 = [VerticalImageLeft(),VerticalImageUp()]
list2 = [HorizontalUp(),HorizontalLeft()]

list_ = list1
for i in range(len(list_)):
    fl,set_pos = list_[i]
    moving_image= clip_image.fl(fl, apply_to=['mask'])
    # 合并视频
    final = CompositeVideoClip([
             bak_movei,
             moving_image.set_pos(set_pos)])

    final.set_duration(5).write_videofile("materials/45岁林志玲身体亮红灯！被曝秘密入院做手术/clean_jpg/test{}.mp4".format(i), fps=20)